# Visualizing DEM differencing data #

import python libraries

In [2]:
import geoutils
import gdal
import georaster
import pandas
from matplotlib import pyplot as plt

//anaconda/envs/py27/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
